In [1]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt

In [4]:
ls /ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/ | grep data

LSFF_extraction_clean_data_rich_locations_01_11_2021.csv
LSFF_extraction_clean_data_rich_locations_notes.xlsx
data_counts_full.csv
data_counts_full_3_10_2021.csv
data_counts_full_3_11_2021.csv
data_counts_full_3_8_2021.csv
data_counts_ideal_bins.csv
data_counts_ideal_bins_3_10_2021.csv
data_counts_ideal_bins_3_11_2021.csv
data_counts_ideal_bins_3_8_2021.csv
data_counts_nutrient_na.csv
data_counts_nutrient_na_3_10_2021.csv
data_counts_nutrient_na_3_11_2021.csv
data_counts_nutrient_na_3_8_2021.csv
data_summary_percent_of_population_eating_fortified_vehicle.csv
data_summary_percent_of_population_eating_industrially_prod_vehicle.csv
data_summary_percent_of_population_eating_vehicle.csv
data_summary_percent_of_vehicle_fortified.csv
data_summary_percent_of_vehicle_industrially_produced.csv
gday_data_assumptions_nigeria_ethiopia_india.xlsx
gday_data_assumptions_tier2.xlsx
lsff_data_estimated.csv
lsff_data_estimated_03_08_2021.csv
lsff_data_estimated_03_11_2021.csv
lsff_data_stage0.csv
lsff_da

In [67]:
est_data_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/population_coverage_data_and_estimates_3_14_2021.csv'
df = pd.read_csv(est_data_path)

In [68]:
df.head()

,location_name,vehicle,nutrient,value_description,value_mean,value_025_percentile,value_975_percentile,is_estimate,sub_population,source_year,standard
0,Angola,maize flour,folic acid,percent of vehicle that is fortified,0.0,NaN,NaN,0.0,total population,2018,NaN
1,Angola,maize flour,iron,percent of vehicle that is fortified,0.0,NaN,NaN,0.0,total population,2018,NaN
2,Angola,maize flour,vitamin b1,percent of vehicle that is fortified,0.0,NaN,NaN,0.0,total population,2018,NaN
3,Angola,maize flour,vitamin b12,percent of vehicle that is fortified,0.0,NaN,NaN,0.0,total population,2018,NaN
4,Angola,maize flour,zinc,percent of vehicle that is fortified,0.0,NaN,NaN,0.0,total population,2018,NaN


In [69]:
val_map = {
    'percent of population eating industrially produced vehicle':'B',
       'percent of population eating vehicle':'A',
       'percent of population eating fortified vehicle':'C'
}
df.value_description = df.value_description.map(val_map)

In [70]:
df = df[df.value_description.notna()]

In [71]:
var = "A"
tmp = df.loc[(df.value_description==var),['location_name','vehicle','value_mean']].rename(columns={'value_mean':var})

var = "B"
r = df.loc[(df.value_description==var),['location_name','vehicle','value_mean']].rename(columns={'value_mean':var})
tmp= tmp.merge(r, on = ['location_name','vehicle'], how = 'outer')

var = 'C'
r = df.loc[(df.value_description==var),['location_name','vehicle','nutrient','value_mean']].rename(columns={'value_mean':var})
tmp= tmp.merge(r, on = ['location_name','vehicle'], how = 'outer')

In [72]:
df = tmp[['location_name','vehicle','nutrient','A','B','C']]

In [73]:
df

,location_name,vehicle,nutrient,A,B,C
0,Bangladesh,oil,vitamin a,95.0,88.40,42.849572
1,Bangladesh,oil,vitamin d,95.0,88.40,0.000000
2,Bangladesh,oil,vitamin a,95.0,87.50,42.849572
3,Bangladesh,oil,vitamin d,95.0,87.50,0.000000
4,Bangladesh,oil,vitamin a,95.0,87.95,42.849572
...,...,...,...,...,...,...
4979,Indonesia,wheat flour,iron,NaN,NaN,73.100000
4980,Indonesia,wheat flour,vitamin b1,NaN,NaN,73.100000
4981,Indonesia,wheat flour,zinc,NaN,NaN,73.100000
4982,Niger,wheat flour,folic acid,NaN,NaN,30.000000


In [74]:
location_names = ["Burkina Faso", "Kenya", "Myanmar", "Vietnam", "Nepal"]
vehicles = ["maize flour","wheat flour","oil"]
nutrients = ["vitamin a","zinc","folic acid","iron"]

In [75]:
tier3 = df[df.location_name.isin(location_names) & (df.vehicle.isin(vehicles)) & (df.nutrient.isin(nutrients))].sort_values('location_name')
tier3

,location_name,vehicle,nutrient,A,B,C
608,Burkina Faso,maize flour,iron,0.2,0.00,0.000000
960,Burkina Faso,wheat flour,folic acid,57.0,52.30,32.164500
959,Burkina Faso,wheat flour,iron,57.0,52.30,17.389750
958,Burkina Faso,wheat flour,vitamin a,57.0,52.30,0.000000
955,Burkina Faso,wheat flour,zinc,57.0,52.30,NaN
...,...,...,...,...,...,...
4729,Vietnam,wheat flour,iron,31.1,21.93,4.111875
4730,Vietnam,wheat flour,zinc,31.1,21.93,0.000000
4731,Vietnam,wheat flour,folic acid,31.1,21.93,0.000000
4779,Vietnam,wheat flour,folic acid,16.4,21.93,0.000000


In [61]:
tier3_missing = tier3[tier3[['A','B','C']].isnull().any(1)].drop_duplicates()
tier3_missing

,location_name,vehicle,nutrient,A,B,C
955,Burkina Faso,wheat flour,zinc,57.00,52.3,NaN
943,Burkina Faso,wheat flour,zinc,41.00,52.3,NaN
991,Burkina Faso,wheat flour,zinc,88.00,52.3,NaN
895,Burkina Faso,wheat flour,zinc,83.00,52.3,NaN
871,Burkina Faso,wheat flour,zinc,48.00,52.3,NaN
1087,Burkina Faso,wheat flour,zinc,58.00,52.3,NaN
1093,Burkina Faso,wheat flour,zinc,62.00,52.3,NaN
1027,Burkina Faso,wheat flour,zinc,19.00,52.3,NaN
823,Burkina Faso,wheat flour,zinc,15.00,52.3,NaN
799,Burkina Faso,wheat flour,zinc,52.00,52.3,NaN


In [62]:
tier3_missing.groupby(['location_name','vehicle']).mean()

A     B     C
location_name vehicle                        
Burkina Faso  oil          48.906   NaN  72.5
              wheat flour  52.300  52.3   NaN
Nepal         oil          99.900   NaN   0.0

In [76]:
ls /ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/ | grep regression_estimates

pct_eating_fortified_regression_estimates_3_15_2021.csv
pct_eating_ind_prod_regression_estimates_3_15_2021.csv
pct_eating_regression_estimates_3_15_2021.csv


In [77]:
outputs_dir = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/'
pct_eating = pd.read_csv(outputs_dir + 'pct_eating_regression_estimates_3_15_2021.csv')
pct_ind_prod = pd.read_csv(outputs_dir + 'pct_eating_ind_prod_regression_estimates_3_15_2021.csv')
pct_fortified = pd.read_csv(outputs_dir + 'pct_eating_fortified_regression_estimates_3_15_2021.csv')

In [78]:
estimates = pct_eating.merge(
    pct_ind_prod, on = ['location_name','vehicle'], how = 'outer'
).merge(
    pct_fortified, on = ['location_name','vehicle'], how = 'outer'
)

In [79]:
est_col_rename = {
    'percent_of_population_eating_vehicle':'A_est',
       'percent_of_population_eating_industrially_produced_vehicle':'B_est',
       'percent_of_population_eating_fortified_vehicle':'C_est'
}
estimates = estimates.rename(columns = est_col_rename)

In [81]:
estimates

,location_name,vehicle,A_est,B_est,C_est
0,Bangladesh,oil,95.000000,87.950000,21.424786
1,Bangladesh,oil,95.000000,87.950000,21.424786
2,Bangladesh,oil,89.000000,87.950000,21.424786
3,Bangladesh,oil,89.000000,87.950000,21.424786
4,Bangladesh,oil,75.000000,87.950000,21.424786
...,...,...,...,...,...
219,Indonesia,oil,93.093204,73.711632,21.162153
220,Egypt,oil,68.152554,89.567348,23.667282
221,Angola,oil,94.805664,36.783381,42.656121
222,Madagascar,oil,78.063762,82.798201,20.984240


In [82]:
compare = tier3_missing.merge(estimates, on = ['location_name','vehicle'], how = 'left').drop(columns=['nutrient']).drop_duplicates()

In [85]:
compare[['location_name','vehicle','A','A_est','B','B_est','C','C_est']]

,location_name,vehicle,A,A_est,B,B_est,C,C_est
0,Burkina Faso,wheat flour,57.0,NaN,52.3,NaN,NaN,NaN
1,Burkina Faso,wheat flour,41.0,NaN,52.3,NaN,NaN,NaN
2,Burkina Faso,wheat flour,88.0,NaN,52.3,NaN,NaN,NaN
3,Burkina Faso,wheat flour,83.0,NaN,52.3,NaN,NaN,NaN
4,Burkina Faso,wheat flour,48.0,NaN,52.3,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
157,Burkina Faso,oil,30.2,52.0,NaN,51.975594,75.0,48.333333
160,Nepal,oil,99.8,99.8,NaN,74.747650,0.0,0.000000
162,Nepal,oil,99.8,100.0,NaN,74.747650,0.0,0.000000
164,Nepal,oil,100.0,99.8,NaN,74.747650,0.0,0.000000


In [88]:
estimates

,location_name,vehicle,A_est,B_est,C_est
0,Bangladesh,oil,95.000000,87.950000,21.424786
1,Bangladesh,oil,95.000000,87.950000,21.424786
2,Bangladesh,oil,89.000000,87.950000,21.424786
3,Bangladesh,oil,89.000000,87.950000,21.424786
4,Bangladesh,oil,75.000000,87.950000,21.424786
...,...,...,...,...,...
219,Indonesia,oil,93.093204,73.711632,21.162153
220,Egypt,oil,68.152554,89.567348,23.667282
221,Angola,oil,94.805664,36.783381,42.656121
222,Madagascar,oil,78.063762,82.798201,20.984240


In [89]:
estimates[(estimates.location_name=="Burkina Faso")]

,location_name,vehicle,A_est,B_est,C_est
147,Burkina Faso,oil,92.23,51.975594,48.333333
148,Burkina Faso,oil,92.23,51.975594,48.333333
149,Burkina Faso,oil,92.23,51.975594,48.333333
150,Burkina Faso,oil,30.20,51.975594,48.333333
151,Burkina Faso,oil,30.20,51.975594,48.333333
152,Burkina Faso,oil,30.20,51.975594,48.333333
153,Burkina Faso,oil,16.10,51.975594,48.333333
154,Burkina Faso,oil,16.10,51.975594,48.333333
155,Burkina Faso,oil,16.10,51.975594,48.333333
156,Burkina Faso,oil,54.00,51.975594,48.333333


In [87]:
compare[['location_name','vehicle','A','A_est','B','B_est','C','C_est']].groupby(['location_name','vehicle']).mean()

A   A_est     B      B_est     C      C_est
location_name vehicle                                                      
Burkina Faso  oil          48.906  48.906   NaN  51.975594  72.5  48.333333
              wheat flour  52.300     NaN  52.3        NaN   NaN        NaN
Nepal         oil          99.900  99.900   NaN  74.747650   0.0   0.000000

In [66]:
path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/lsff_estimated_data_plus_covariates_with_nutrient_3_14_2021.csv'
test = pd.read_csv(path)

In [67]:
test = test[test.location_name.isin(location_names) &
            (test.vehicle.isin(vehicles)) &
            (test.nutrient.isin(nutrients))].sort_values('location_name').rename(columns=est_col_rename)


In [71]:
test = test[test[['A_est','B_est','C_est']].isnull().any(1)]

In [74]:
test[['location_name','vehicle','A_est','B_est','C_est']]

,location_name,vehicle,A_est,B_est,C_est
1069,Burkina Faso,oil,52.00,NaN,75.0
1066,Burkina Faso,oil,54.00,NaN,75.0
1065,Burkina Faso,oil,54.00,NaN,70.0
1063,Burkina Faso,oil,16.10,NaN,75.0
1062,Burkina Faso,oil,16.10,NaN,70.0
1060,Burkina Faso,oil,30.20,NaN,75.0
1059,Burkina Faso,oil,30.20,NaN,70.0
1057,Burkina Faso,oil,92.23,NaN,75.0
1056,Burkina Faso,oil,92.23,NaN,70.0
1068,Burkina Faso,oil,52.00,NaN,70.0
